In [15]:
import os
import csv
import schedule
import time
import pandas as pd
from csv import writer
from googleapiclient.discovery import build
from dotenv import load_dotenv

# Set API parameters.
load_dotenv()
API_KEY = os.getenv("API_KEY")
CHANNEL_ID = os.getenv("CHANNEL_ID")
youtube = build('youtube','v3', developerKey = API_KEY)

class ytStats:
    def __init__(self, API_KEY, CHANNEL_ID):
        self.API_KEY = API_KEY
        self.CHANNEL_ID = CHANNEL_ID

    def get_channel_statistics(self):
        all_data = []
        request = youtube.channels().list(
            part = "snippet,contentDetails,statistics",
            id = CHANNEL_ID
            )
        response = request.execute()

        for i in range (len(response['items'])):
            data = dict(
                result_channel_name = response['items'][i]['snippet']['title']  , 
                result_playlist_id = response['items'][i]['contentDetails']['relatedPlaylists']['uploads'] ,
                result_channel_subscribers = response['items'][i]['statistics']['subscriberCount']  , 
                result_channel_views = response['items'][i]['statistics']['viewCount']  , 
                result_channel_total_videos = response['items'][i]['statistics']['videoCount'] 
                )
            all_data.append(data)              
        return all_data

    def get_video_ids(youtube, playlist_id):
        request = youtube.playlistItems().list(
            part = 'contentDetails' , 
            playlistId = playlist_id , 
            maxResults = 50
        )
        response = request.execute()

        all_data = []

        for i in range(len(response['items'])):
            all_data.append(response['items'][i]['contentDetails']['videoId'])

        next_page_token = response.get('nextPageToken')
        more_pages = True

        while more_pages:
            if next_page_token is None:
                more_pages = False
            else:
                request = youtube.playlistItems().list(
                    part = 'contentDetails' , 
                    playlistId = playlist_id , 
                    maxResults = 50 , 
                    pageToken = next_page_token
                )
                response = request.execute()
                
                for i in range(len(response['items'])):
                    all_data.append(response['items'][i]['contentDetails']['videoId'])
                
                next_page_token = response.get('nextPageToken')
        
        return all_data

    def get_video_details(youtube, video_ids):
        all_data = []
        for i in range(0, len(video_ids), 50):
            request = youtube.videos().list(
                part = 'snippet, statistics' , 
                id = ','.join(video_ids[i: i + 50]) ,
                #order = 'date'
            )
            response = request.execute()

            for video in response['items']:
                video_stats = dict(
                    result_video_name = video['snippet']['title']  ,
                    result_video_id = video['id'] ,                
                    result_video_description = video['snippet']['description']  ,
                    result_video_upload_time = video['snippet']['publishedAt']  ,
                    result_video_views = video['statistics']['viewCount'] ,  
                    result_video_likes = video['statistics']['likeCount'] ,  
                    result_video_favorites = video['statistics']['favoriteCount'] ,  
                    result_video_comments = video['statistics']['commentCount']  
                    )
                all_data.append(video_stats)

        return all_data

In [16]:
# Call functions
yt = ytStats
channel_statss = yt.get_channel_statistics(youtube)#, channel_id)
playlist_id    = channel_statss[0]['result_playlist_id']
video_ids      = yt.get_video_ids(youtube, playlist_id)                # Time = 5.6s
video_details  = yt.get_video_details(youtube, video_ids)              # Time = 8.5s

In [17]:
video_details

[{'result_video_name': 'When a bullet shoots through the earth',
  'result_video_id': 'QlA8J3elpYg',
  'result_video_description': "It's GunSpin.\n• Coffee - https://floridamancoffeeco.com/\n• Merch - https://www.graystillplays.store/\n• Facebook - https://www.facebook.com/graystillplays/\n• Instagram - https://www.instagram.com/graystillplaysofficial/?hl=en\n• TikTok - https://www.tiktok.com/@graystillplaystiktok?lang=en\n• Twitter - https://twitter.com/GrayStillPlays\n• Join this channel to get access to perks:\nhttps://www.youtube.com/channel/UCzEy7pi3B7TIS9cn_sdKK9A/join\n----------------------------------------------------------------\nWatch more Graystillplays\n• Games Where You Make Terrible Choices - https://www.youtube.com/watch?v=sg0drcURlMQ&list=PLw8xXEJ0p21cRkni_fMVQpDsHZt3rCo-C\n----------------------------------------------------------------\nABOUT - GunSpin\nGunspin is a physics game where you shoot a gun to keep it flying for as long as possible. Upgrade your gun with m